In [1]:
import os

import numpy as np
import pandas as pd

from utils import load_hidden_representations_from_hdf5, read_templates_from_file, read_predictions_from_file

In [2]:
# params
log_dir = "/logfiles"
model = "bigscience-T0"
module = "decoder"
task = "rte"

In [3]:
df = read_templates_from_file(f"/t0-analysis/prompts/{task}/all.csv")
# df = read_templates_from_file(f"/t0-analysis/prompts/{task}/fixed_prompt.csv")
# df = read_templates_from_file(f"/t0-analysis/prompts/{task}/fixed_target_yes_no.csv")
display(df)

,name,template,category,includes_targets,targets,target_ids,shuffle
0,gpt_3_yes_no_with_targets,{premise} Question: {hypothesis} Yes or No?,instructive,True,"▁Yes, ▁No","0, 1",False
1,mnli_crowdsource_with_targets,{premise} Using only the above description and...,instructive,True,"▁Yes, ▁No","0, 1",False
2,based_on_previous_passage_with_targets,"{premise} Based on the previous passage, is it...",instructive,True,"▁Yes, ▁No","0, 1",False
3,infer_with_targets,"Suppose {premise} Can we infer that ""{hypothes...",instructive,True,"▁Yes, ▁No","0, 1",False
4,follow_with_targets,Given that {premise} Does it follow that {hypo...,instructive,True,"▁Yes, ▁No","0, 1",False
5,imply_with_targets,"{premise} Question: Does this imply that ""{hyp...",instructive,True,"▁Yes, ▁No","0, 1",False
6,guaranteed_with_targets,"Given {premise} Is it guaranteed true that ""{h...",instructive,True,"▁Yes, ▁No","0, 1",False
7,justified_with_targets,"{premise} Are we justified in saying that ""{hy...",instructive,True,"▁Yes, ▁No","0, 1",False
8,must_be_true_with_targets,"Given that {premise} Therefore, it must be tru...",instructive,True,"▁Yes, ▁No","0, 1",False
9,should_assume_with_targets,"Given {premise} Should we assume that ""{hypoth...",instructive,True,"▁Yes, ▁No","0, 1",False


In [4]:
# load prompted inputs and labels
df_inputs = {}
for _, row in df.iterrows():
    prompt = row['name']
    df_inputs[prompt] = pd.read_csv(f"/logfiles/{task}/{model}/{module}/{prompt}/prompted_samples.csv", sep='\t', index_col=0)


In [5]:
for _, row in df.iterrows():
    prompt = row["name"]
    print(prompt)
    for idx, row in df_inputs[prompt].head(10).iterrows():
        print(idx, row["input"], row["label"], row["prediction"])
    


gpt_3_yes_no_with_targets
0 Dana Reeve, the widow of the actor Christopher Reeve, has died of lung cancer at age 44, according to the Christopher Reeve Foundation. Question: Christopher Reeve had an accident. Yes or No? not_entailment not_entailment
1 Yet, we now are discovering that antibiotics are losing their effectiveness against illness. Disease-causing bacteria are mutating faster than we can come up with new antibiotics to fight the new variations. Question: Bacteria is winning the war against antibiotics. Yes or No? entailment not_entailment
2 Cairo is now home to some 15 million people - a burgeoning population that produces approximately 10,000 tonnes of rubbish per day, putting an enormous strain on public services. In the past 10 years, the government has tried hard to encourage private investment in the refuse sector, but some estimate 4,000 tonnes of waste is left behind every day, festering in the heat as it waits for someone to clear it up. It is often the people in the